<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
import time
import os
import pandas as pd
import stlabutils

from src.model_currentbias import lossrate

from scipy.constants import hbar, e, h, Planck, c

Phi0 = Planck/(2*e)
pi = np.pi
c0 = c

In [2]:
import glob
import stlabutils
simfiles = sorted(glob.glob('Duffing/JJarrayCPW*.dat'))

In [3]:
from src.algo_inputoutput import first_order_coeffs, nphot_to_a0
from src.algo_peakheights import func_PdBmtoV, func_cmpx_to_dBm
import glob
from scipy.interpolate import interp1d

In [4]:
power_gain = pickle.load(
    open("data_processed/sensitivity_estimation/gaindata.pkl", "rb"))
gain = power_gain['Gain (dB)']

In [5]:
for simfile_min in simfiles:
    print('Importing from ',simfile_min)
    simdata_min = stlabutils.readdata.readdat(simfile_min)

    fixcurrent = simdata_min[0]['Iset (A)'][0]
    numcurrent = round(int(fixcurrent/1e-9), -1)

    l = simdata_min[0]['l (m)'][0]
    N = simdata_min[0]['N ()'][0]

    fpump = simdata_min[0]['f0 (Hz)'][0]
    G1 = simdata_min[0]['G1 (Hz/A)'][0]
    kext = simdata_min[0]['kext (Hz)'][0]
    kint = simdata_min[0]['kint (Hz)'][0]
    ktot = kext+kint

    pklpath = 'data_processed/Duffing/'+simfile_min.split('/')[-1].strip('.dat')+'.pkl'
    print('Exporting to ',pklpath)
#     if not os.path.isfile(pklpath):
    if True:

        # From the simulation, extracting actual photon number at pump frequency
        nph_int = []
        nph_int_max = []
        pwr = []
        fsim_min = []

        for simblock in simdata_min:
            thepower = simblock['Probe power (dBm)'].iloc[0]
            nph = simblock['Intracavity photons ()'].values
            freq = simblock['Probe Frequency (Hz)'].values
            yy = simblock['S11dB (dB)'].values
            idx = yy.argmin()
            fsim_min.append(freq[idx])
            nph_int.append(interp1d(freq, nph)(fpump))
            nph_int_max.append(interp1d(freq, nph)(freq[idx]))
            pwr.append(thepower)
        nph_int = np.array(nph_int)
        nph_int_max = np.array(nph_int_max)
        pwr = np.array(pwr)
        # converting intracavity photon number to voltage amplitude
        alpha_0 = nphot_to_a0(nph_int, fpump)
        fsim_min = np.array(fsim_min)
        alpha_0_max = nphot_to_a0(nph_int_max, fsim_min)

        # Defining the individual terms to match the variable names in the formulas
        iac = 10e-9
        Omega = 1e6

        # corrections compared to non-Duffing analysis
        # Using the same cavity parameters as for the simulation
        beta = simdata_min[0]['Beta (Hz)'][0]
        fres_Duffing = fpump+2*beta*nph_int
        Delta_Duffing = fpump-fres_Duffing

        """
        ************************************************************************************
        ************************************************************************************
        ************************************************************************************
        First order calculation
        ************************************************************************************
        ************************************************************************************
        ************************************************************************************
        """

        # calculating the first order peak coefficients heights
        cm1, _ = first_order_coeffs(
            iac, Omega, G1, kext, ktot, Sin=0, Delta=Delta_Duffing, a0_calc=alpha_0)
        Sm1_dBm = func_cmpx_to_dBm(cm1, kext, gain)

        # same but without detuning
        cm1_nodet, _ = first_order_coeffs(
            iac, Omega, G1, kext, ktot, Sin=0, Delta=0, a0_calc=alpha_0_max)
        Sm1_dBm_nodet = func_cmpx_to_dBm(cm1_nodet, kext, gain)

        # save for figure
        simpkl = {'xtheo': pwr, 'ytheo': Sm1_dBm, 'ytheolabel': 'model',
                  'xtheo2': pwr, 'ytheo2': Sm1_dBm_nodet, 'ytheolabel2': 'model on resonance',
                  'xlabel': 'Power at sample (dBm)', 'ylabel': 'Amplitude (dBm)',
                  'Iset (A)': fixcurrent, 'Beta (Hz)': beta,
                 'nphot':nph_int, 'nphot2':nph_int_max,'l':l,'N':N}
        pickle.dump(simpkl, open(pklpath, 'wb'))

Importing from  Duffing/JJarrayCPW_1.0_845_0000.dat
['Probe Frequency (Hz)', 'Probe power (W)', 'Probe power (dBm)', 'S11dB (dB)', 'S11ph (rad)', 'Intracavity photons ()', 'Beta (Hz)', 'Delta (Hz)', 'Phi (rad)', 'Iset (A)', 'kint (Hz)', 'kext (Hz)', 'f0 (Hz)', 'G1 (Hz/A)', 'l (m)', 'N ()']
Exporting to  data_processed/Duffing/JJarrayCPW_1.0_845_0000.pkl
Importing from  Duffing/JJarrayCPW_1.0_845_0100.dat
['Probe Frequency (Hz)', 'Probe power (W)', 'Probe power (dBm)', 'S11dB (dB)', 'S11ph (rad)', 'Intracavity photons ()', 'Beta (Hz)', 'Delta (Hz)', 'Phi (rad)', 'Iset (A)', 'kint (Hz)', 'kext (Hz)', 'f0 (Hz)', 'G1 (Hz/A)', 'l (m)', 'N ()']
Exporting to  data_processed/Duffing/JJarrayCPW_1.0_845_0100.pkl
Importing from  Duffing/JJarrayCPW_1.0_845_0200.dat
['Probe Frequency (Hz)', 'Probe power (W)', 'Probe power (dBm)', 'S11dB (dB)', 'S11ph (rad)', 'Intracavity photons ()', 'Beta (Hz)', 'Delta (Hz)', 'Phi (rad)', 'Iset (A)', 'kint (Hz)', 'kext (Hz)', 'f0 (Hz)', 'G1 (Hz/A)', 'l (m)', 'N ()